# Helper functions for Tensorflow Fold



In [ ]:
length = td.Length()

embedded =  (td.InputTransform(lambda s: [index(x) for x in s]) >> 
             td.Map(td.Scalar(tf.int32) >> 
             td.Function(td.Embedding(vsize, EMBEDDING_SIZE))))

onehot = (td.InputTransform(lambda s: [index(x) for x in s]) >>
          td.Map(td.Scalar(tf.int32) >>
          td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
          td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))

decode_onehot = td.InputTransform(lambda s: [char(np.argmax(np.squeeze(x))) for x in s])

print(decode_onehot.eval(onehot.eval("malú")))
#embedded.eval("kacsa")

In [ ]:
def conv1d_on_sequence(x, scope, kernel_size=3, input_channels=72, output_channels=72):
    with tf.variable_scope(scope) as sc:
        filters = tf.get_variable("conv_filter", [kernel_size] +  [input_channels, output_channels] , initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable("conv_bias",  output_channels, initializer=tf.constant_initializer(0.05, dtype=tf.float32))
        conv = tf.nn.conv1d(x, filters=filters, stride=1, padding='VALID')
        return tf.nn.relu(tf.add(conv, bias))
    
def SeqToTuple(T, N):
    return (td.InputTransform(lambda x: tuple(x))
            .set_input_type(td.SequenceType(T))
            .set_output_type(td.Tuple(*([T] * N))))   